cnews 中文文本分类：     
由清华大学根据新浪新闻RSS订阅频道2005-2011年间的历史数据筛选过滤生成     
训练集 50000    
验证集 5000     
测试集 10000    
词汇（字） 5000      
10个分类，包括：'体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")


In [3]:
!/opt/bin/nvidia-smi

Tue Oct 27 14:40:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null !apt-get update -qq 2>&1 > /dev/null !apt-get -y install -qq google-drive-ocamlfuse fuse from google.colab import auth auth.authenticate_user() from oauth2client.client import GoogleCredentials creds = GoogleCredentials.get_application_default() import getpass !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} &1 | grep URL vcode = getpass.getpass() !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `apt-get install -y -qq software-properties-common python-software-properties module-init-tools !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null !apt-get update -qq 2>&1 > /dev/null !apt-get -y install -qq google-drive-ocamlfuse fuse from google.colab import auth auth.authenticate_user() from oauth2client.client import GoogleCredentials creds = GoogleCredentials.get_application_default() import getpass !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} &1 | grep URL vcode = getpass.getpass() !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}'


In [5]:
import numpy as np
from tensorflow import keras as kr

In [6]:
import torch 
from torch import nn
from torch.nn import functional as F

In [7]:
import numpy as np

In [8]:
from torch.utils.data import DataLoader, TensorDataset

In [9]:
from tqdm.notebook import tnrange

# 1 准备数据——将文本转为数值向量


---
```
keras.preprocessing.sequence.pad_sequences(sequences, 
	maxlen=None,
	dtype='int32',
	padding='pre',
	truncating='pre', 
	value=0.)```    
- sequences：浮点数或整数构成的两层嵌套列表
- maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
- dtype：返回的numpy array的数据类型
- padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补`
- truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
- value：浮点数，此值将在填充时代替默认的填充值0

In [10]:
# 读取词汇表
def read_vocab(vocab_dir):
    with open(vocab_dir, 'r', encoding='utf-8', errors='ignore') as fp:
        # 读取进来的列表中，每个元素为str，str最后包含换行符都不可见空白，所以需要用strip去掉
        words = [_.strip() for _ in fp.readlines()]
    # 建立每段的文本和它的下标的映射字典
    word_to_id = dict(zip(words, range(len(words))))
    return words, word_to_id


# 读取分类目录，固定
def read_category():
    categories = ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
    # 建立分类与下标的映射字典
    cat_to_id = dict(zip(categories, range(len(categories))))
    return categories, cat_to_id


# 将文件转换为id表示
def process_file(filename, word_to_id, cat_to_id, max_length=600):
    contents, labels = [], []
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            try:
                label, content = line.strip().split('\t')
                if content:
                    # 句子转为单个字符以列表形式存入contents
                    contents.append(list(content))
                    labels.append(label)
            except:
                pass
    data_id, label_id = [], []
    for i in range(len(contents)):
        #将每句话id化
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        #每句话对应的类别的id
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    # 将整数列表转换成形状为(samples, max_length) 的二维整数张量
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, max_length)
    y_pad = kr.utils.to_categorical(
        label_id, num_classes=len(cat_to_id))  # 将标签转换为one-hot表示
    return x_pad, y_pad

在训练集小样本文件中，最长的句子是第120个，有10337个字符。

## 获取文本的类别及其对应id的字典

In [11]:
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()

In [12]:
print(categories)
print(cat_to_id)

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
{'体育': 0, '财经': 1, '房产': 2, '家居': 3, '教育': 4, '科技': 5, '时尚': 6, '时政': 7, '游戏': 8, '娱乐': 9}


## 获取训练文本中所有出现过的字及其所对应的id

In [13]:
fpath = './cnews/cnews.vocab.txt'

# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab(fpath)

In [14]:
#获取字数
vocab_size = len(words)

In [15]:
# 说明有重复文字
vocab_size, len(word_to_id)

(5000, 4998)

## 加载训练集、验证集、测试集

In [16]:
# os.listdir('./cnews')

In [17]:
# 获取训练数据每个字的id和对应label的one-hot形式
sample_train_path = './cnews/cnews.train.txt'
x_train, y_train = process_file(sample_train_path, word_to_id, cat_to_id, 600)
print('x_train=', x_train)

x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [18]:
# 读取验证集
val_path = './cnews/cnews.val.txt'
x_val, y_val = process_file(val_path, word_to_id, cat_to_id, 600)

In [19]:
# 读取测试集
test_path = './cnews/cnews.test.txt'
x_test, y_test = process_file(test_path, word_to_id, cat_to_id, 600)

In [20]:
# 训练集、验证集、测试集全部转为张量
x_train = torch.tensor(x_train, dtype=torch.int64)

y_train = torch.tensor(y_train, dtype=torch.float32)

x_val = torch.tensor(x_val, dtype=torch.int64)

y_val = torch.tensor(y_val, dtype=torch.float32)

x_test = torch.tensor(x_test, dtype=torch.int64)

y_test = torch.tensor(y_test, dtype=torch.float32)

In [21]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

(torch.Size([50000, 600]),
 torch.Size([50000, 10]),
 torch.Size([5000, 600]),
 torch.Size([5000, 10]),
 torch.Size([10000, 600]),
 torch.Size([10000, 10]))

In [22]:
x_train.dtype, y_train.dtype

(torch.int64, torch.float32)

## 数据分批

In [23]:
trainset = TensorDataset(x_train, y_train)

In [24]:
BATCH_SIZE = 128
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

In [25]:
len(trainloader)

391

In [26]:
testset = TensorDataset(x_test, y_test)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

In [27]:
valset = TensorDataset(x_val, y_val)
valloader = DataLoader(valset, batch_size=BATCH_SIZE)

In [28]:
len(testloader), len(valloader)

(79, 40)

# 2 自定义神经网络

In [29]:
# 输入形状是语料库的单词数
vocab_size

5000

In [30]:
# 文本分类，RNN模型
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        # 词向量嵌入，输入2维，输出3维，最后一维为64
        self.embedding = nn.Embedding(5000, 64)  # 进行词嵌入
        # GRU需要2个输入，
        # batch_first=True时，input_0 of shape (batch, seq_len, input_size)
        # h_0 of shape (batch, num_layers * num_directions, hidden_size)
        self.rnn = nn.GRU(input_size=64,
                          hidden_size=128,
                          num_layers=2,
                          batch_first=True,
                          bidirectional=True)
        self.f1 = nn.Sequential(nn.Linear(256, 128), nn.Dropout(0.8),
                                nn.ReLU())
        self.f2 = nn.Sequential(nn.Linear(128, 10), nn.Softmax(1))

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = F.dropout(x, p=0.8)
        # 取最后一个时间步
        x = self.f1(x[:, -1, :])
        return self.f2(x)  # (batch_size, 10)
    

# 建模

## 实例化模型

In [36]:
textrnn = TextRNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
textrnn.to(device)

TextRNN(
  (embedding): Embedding(5000, 64)
  (rnn): GRU(64, 128, num_layers=2, batch_first=True, bidirectional=True)
  (f1): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): Dropout(p=0.8, inplace=False)
    (2): ReLU()
  )
  (f2): Sequential(
    (0): Linear(in_features=128, out_features=10, bias=True)
    (1): Softmax(dim=1)
  )
)

## 编译模型

In [37]:
LR = 1e-3
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(textrnn.parameters(), lr=LR)

## 训练

In [38]:
EPOCH = 20
costs = []

early_stop = 0
min_loss = float('inf')
best_acc = 0.

for epoch in tnrange(EPOCH):
    losses = []
    train_correct, train_total = 0., 0.
    for step, (b_x, b_y) in enumerate(trainloader):
        b_x, b_y = b_x.to(device), b_y.to(device)
        # 前向传播
        output = textrnn(b_x)
        # 计算loss
        loss = criterion(output, b_y)
        losses.append(loss.item())
        # 梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 权值更新
        optimizer.step()
        # 统计训练集准确数量
        train_pred = torch.argmax(output, 1).data.cpu().numpy()
        train_label = torch.argmax(b_y, 1).data.cpu().numpy()
        train_correct += (train_pred == train_label).sum()
        train_total += len(train_label)
    meanloss = np.mean(losses)
    costs.append(meanloss)
    if epoch % 2 == 0:
        # 训练集准确率
        train_acc = train_correct / train_total
        print("==>epoch:{} 训练集loss:{:.4f} 训练集accuracy:{:.2f}% ".format(epoch, meanloss, train_acc * 100))
        # textrnn.train(False)
        # 验证集预测
        # val_correct, val_total = 0., 0.
        # for i, (x_v, y_v) in enumerate(valloader):
        #     x_v, y_v = x_v.to(device), y_v.to(device)
        #     val_output = textrnn(x_v.to(device))
        #     # 获取预测的label，并转为数组
        #     val_pred = torch.argmax(val_output, 1)
        #     val_pred_arr = val_pred.data.cpu().numpy()
        #     y_val_arr = torch.argmax(y_v, 1).data.cpu().numpy()
        #     # 准确率
        #     val_correct += (val_pred_arr == y_val_arr).sum()
        #     val_total += len(y_val_arr)
        # val_acc = val_correct / val_total
        # print("==>epoch:{} 训练集loss:{:.4f} 训练集accuracy:{:.2f}% 验证集accuracy:{:.2f}%".
        #     format(epoch, meanloss, train_acc * 100, val_acc * 100))
        # textrnn.train(True)
    # 根据准确率保存模型
    # if val_acc > best_acc:
    #     best_acc = val_acc
    #     torch.save(textrnn, 'textRNN.pt')
    #  早停法
    if meanloss < min_loss:
        min_loss = meanloss
        early_stop = 0
    else:
        early_stop += 1
    if early_stop > 5:
        print(f"loss连续{epoch}个epoch未降低, 停止循环")
        break

==>epoch:0 训练集loss:0.7159 训练集accuracy:36.51% 
==>epoch:2 训练集loss:0.6896 训练集accuracy:65.28% 
==>epoch:4 训练集loss:0.6774 训练集accuracy:77.83% 
==>epoch:6 训练集loss:0.6718 训练集accuracy:83.39% 
==>epoch:8 训练集loss:0.6691 训练集accuracy:86.05% 
==>epoch:10 训练集loss:0.6667 训练集accuracy:88.43% 
==>epoch:12 训练集loss:0.6663 训练集accuracy:88.85% 
==>epoch:14 训练集loss:0.6651 训练集accuracy:90.04% 
==>epoch:16 训练集loss:0.6653 训练集accuracy:89.90% 
==>epoch:18 训练集loss:0.6638 训练集accuracy:91.36% 



# 测试集

In [39]:
textrnn.eval()
correct, total = 0, 0
for x_t, y_t in testloader:
    x_t, y_t = x_t.to(device), y_t.to(device)
    output = textrnn(x_t)
    test_pred = torch.argmax(output, 1).data.cpu().numpy()
    test_label = torch.argmax(y_t, 1).data.cpu().numpy()
    correct += (test_pred == test_label).sum()
    total += len(y_t)
    
print('10000个测试集的准确率:{:.4f}'.format(100 * correct / total))

10000个测试集的准确率:90.4100
